# RUN FIRST

In [1]:
# Importing libraries. 

import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import time 

# To avoid non-essential warnings 
import warnings
warnings.filterwarnings('ignore')

from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

from torchvision.models import vgg16, VGG16_Weights


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Preview one of the images..
def show_image(img, label):
    plt.figure(figsize=(1,1))
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1,2,0))

In [3]:
 # To check wether Google Colab GPU has been assigned/not. 

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return None
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [4]:
device = get_default_device()
device 

device(type='cuda')

In [5]:
def acc_and_loss(model,loader,opt):

        model.eval()
        losses = []
        acc_lst = []
        mean_loss,acc = 0,0
        lbl_sz =0
        # criterion = cpt
        optimizer = opt
        #NO gradient required as not training
        with torch.no_grad():
            for i, data in enumerate(loader, 0):
                # get the inputs
                inputs, labels = data[0].to(get_default_device()), data[1].to(get_default_device())
                outputs = model(inputs)
                #Accuracy
                lbl_sz+=labels.size(0)
                _, predicted = torch.max(outputs.data, 1)
                itm = (predicted == labels).sum().item()
                acc_lst.append(itm)
                #losses
                loss = F.cross_entropy(outputs, labels)
                losses.append(loss.item())
        mean_loss = np.mean(losses)
        acc=(100 * np.sum(acc_lst) / lbl_sz)
        return  acc,mean_loss

def fit_A(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD): #this works
  n_total_steps = len(train_loader)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
          # origin shape: [4, 3, 32, 32] = 4, 3, 1024
          # input_layer: 3 input channels, 6 output channels, 5 kernel size
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = F.cross_entropy(outputs, labels)

          # Backward and optimize
          opt_func.zero_grad()
          loss.backward()
          opt_func.step()

          if (i+1) % 2000 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      result_t = acc_and_loss(model,train_loader,opt_func)
      result_v = acc_and_loss(model,val_loader,opt_func)
      print(f"Validation : {result_v}")
      print(f"training : {result_t}")
  print('Finished Training')


def show_graph(epochs,model):
   train_accuracies = [x['train_acc'] for x in model.train_stats]
   val_accuracies = [x['val_acc'] for x in model.train_stats]  
   train_losses = [x['train_loss'] for x in model.train_stats]
   val_losses = [x['val_loss'] for x in model.train_stats] 
   print(train_accuracies)
   
   plt.figure(figsize=(5, 4))
   plt.title("Plotting Training/Validation Loss vs Epochs")
   plt.plot(np.arange(epochs), val_losses, label = "Validation Loss")
   plt.plot(np.arange(epochs), train_losses, label = "Training Loss")
   plt.legend()
   plt.show()
   
   plt.figure(figsize=(5, 4))
   plt.title("Plotting Training/Validation Accuracy  vs Epochs")
   plt.plot(np.arange(epochs), val_accuracies, label = "Validation Accuracy")
   plt.plot(np.arange(epochs), train_accuracies, label = "Training Accuracy")
   plt.legend()
   plt.show()

def acc_by_class(batch_size,model, val_loader):
  with torch.no_grad():
      n_correct = 0
      n_samples = 0
      n_class_correct = [0 for i in range(10)]
      n_class_samples = [0 for i in range(10)]
      for images, labels in val_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          # max returns (value ,index)
          _, predicted = torch.max(outputs, 1)
          n_samples += labels.size(0)
          n_correct += (predicted == labels).sum().item()
          
          for i in range(batch_size):
              label = labels[i]
              pred = predicted[i]
              if (label == pred):
                  n_class_correct[label] += 1
              n_class_samples[label] += 1

      acc = 100.0 * n_correct / n_samples
      print(f'Accuracy of the network: {acc} %')

      for i in range(10):
          acc = 100.0 * n_class_correct[i] / n_class_samples[i]
          print(f'Accuracy of {classes[i]}: {acc} %')

# Experiment 0

In [6]:
# Model print

vgg_weights = VGG16_Weights.DEFAULT
vgg_transforms = vgg_weights.transforms()
print(vgg_transforms)
model = vgg16(weights=vgg_weights, progress=True)
        
model.to(device)
print(model)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# Checking the dataset training size.
dataset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=vgg_transforms)

print('Size of training dataset :', len(dataset))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Size of training dataset : 50000


In [8]:
# Setting seed so that value won't change everytime. 
# Splitting the dataset to training, validation, and testing category.
torch.manual_seed(10)
val_size = len(dataset)//20
test_size = len(dataset)//10
train_size = len(dataset) - val_size - test_size

# Random Splitting. 
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
len(train_ds), len(val_ds),len(test_ds)  

(42500, 2500, 5000)

In [9]:
batch_size = 32 #change batch_size
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)
test_loader = DataLoader(test_ds, batch_size)

In [10]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)
classes = dataset.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [11]:
img, label = dataset[100]
print(img.shape)

input_size = 3*224*224
output_size = 10

torch.Size([3, 224, 224])


In [12]:
# Hints: The following parameters can be changed to improve the accuracy
print(test_size)
num_epochs = 3 #5
lr = 0.001 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9))#


5000


In [13]:
start = time.time()
fit_A(num_epochs, lr, model, train_loader, val_loader, optimizer)
print(time.time()-start)

KeyboardInterrupt: ignored

In [ ]:
show_graph(num_epochs,model)

In [ ]:
acc_by_class(batch_size,model, val_loader)

##FLOPs

In [ ]:
  #The code from https://cloudstor.aarnet.edu.au/plus/s/PcSc67ZncTSQP0E can be used to count flops
  #Download the code.
  !wget -c https://cloudstor.aarnet.edu.au/plus/s/hXo1dK9SZqiEVn9/download
  !mv download FLOPs_counter.py
  #!rm -rf download

In [ ]:
from FLOPs_counter import print_model_parm_flops
input = torch.randn(1, 3, 224, 224) # The input size should be the same as the size that you put into your model 
#Get the network and its FLOPs
num_classes = len(classes)
model = ConvolutionalNetwork(num_classes)
print_model_parm_flops(model, input, detail=False)